In [33]:
import pymysql
from pymysql.cursors import DictCursor
from datetime import timedelta

In [34]:
conn=pymysql.connect(host="localhost",
                            database="digital_factory_v1_completed",
                            user="root",
                            password="")
cur=conn.cursor(DictCursor)


In [41]:
cur.execute(f"select * from digital_factory_v1.master_shifts")
val=cur.fetchall()
val

[{'shift_id': 1,
  'company_id': 1,
  'no_of_shifts': 3,
  'shift1_start_time': datetime.datetime(1900, 1, 1, 7, 0),
  'shift2_start_time': datetime.datetime(1900, 1, 1, 15, 30),
  'shift3_start_time': datetime.datetime(1900, 1, 1, 23, 30),
  'mill_date': datetime.datetime(2022, 8, 3, 0, 0),
  'mill_shift': '3',
  'created_on': datetime.datetime(2019, 3, 28, 21, 35, 14),
  'created_by': 0,
  'modified_on': '2020-08-04 18:41:32',
  'modified_by': 1}]

In [42]:
mill_shift=int(val[0]["mill_shift"])-1
mill_date=val[0]["mill_date"]
# print(mill_shift)
if mill_shift==0:
    mill_shift=val[0]["no_of_shifts"]
    mill_date-=timedelta(1)

print(mill_date)
print(mill_shift)

2022-08-03 00:00:00
2


In [43]:
mill_month={1:"01",2:"02",3:"03",4:"04",5:"05",6:"06",7:"07",8:"08",9:"09",10:"10",11:"11",12:"12"}
month=mill_month[mill_date.month]
shift_table_name="production_"+month+str(mill_date.year)+"_"+"shift"
day_table_name="production_"+month+str(mill_date.year)+"_"+"date"
month_table_name="production_"+month+str(mill_date.year)+"_"+"month"
mill_date=str(mill_date).split()[0]
print(shift_table_name)
print(day_table_name)
print(month_table_name)
print(str(mill_date))


production_082022_shift
production_082022_date
production_082022_month
2022-08-03


In [44]:
cur.execute(f"select exists (select table_name from information_schema.tables where table_schema='digital_factory_v1_completed' and table_name='{shift_table_name}' )as result")
state=cur.fetchall()
state[0]["result"]



1

In [45]:
month

'08'

In [46]:
if state[0]["result"]==0:
	cur.execute(f"""CREATE TABLE {shift_table_name} (
  `id` INT(11) PRIMARY KEY AUTO_INCREMENT AUTO_INCREMENT,
  `department_id` INT(11) NOT NULL DEFAULT 0,
  `machine_id` INT(11) NOT NULL DEFAULT 0,
  `mill_date` DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
  `mill_shift` ENUM('1','2','3') NOT NULL DEFAULT '1',
  `AR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `PR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `QR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `OEE` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `pr_by_val` DOUBLE(21,4) NOT NULL DEFAULT 0.0000,
  `target` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `actual` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `sq_mtr` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `ok_qty` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `running_time` DECIMAL(32,0) NOT NULL DEFAULT 0,
  `loss_time` DECIMAL(33,0) NOT NULL DEFAULT 0
)""")
	conn.commit()
	cur.execute(f"""CREATE TABLE {day_table_name} (
  `id` INT(11) PRIMARY KEY AUTO_INCREMENT AUTO_INCREMENT,
  `department_id` INT(11) NOT NULL DEFAULT 0,
  `machine_id` INT(11) NOT NULL DEFAULT 0,
  `mill_date` DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
  `mill_shift` ENUM('1','2','3') NOT NULL DEFAULT '1',
  `AR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `PR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `QR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `OEE` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `pr_by_val` DOUBLE(21,4) NOT NULL DEFAULT 0.0000,
  `target` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `actual` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `sq_mtr` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `ok_qty` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `running_time` DECIMAL(32,0) NOT NULL DEFAULT 0,
  `loss_time` DECIMAL(33,0) NOT NULL DEFAULT 0
)""")
	conn.commit()
	cur.execute(f"""CREATE TABLE {month_table_name} (
  `id` INT(11) PRIMARY KEY AUTO_INCREMENT AUTO_INCREMENT,
  `department_id` INT(11) NOT NULL DEFAULT 0,
  `machine_id` INT(11) NOT NULL DEFAULT 0,
  `mill_date` DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
  `mill_shift` ENUM('1','2','3') NOT NULL DEFAULT '1',
  `AR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `PR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `QR` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `OEE` DOUBLE(18,1) NOT NULL DEFAULT 0.0,
  `pr_by_val` DOUBLE(21,4) NOT NULL DEFAULT 0.0000,
  `target` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `actual` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `sq_mtr` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `ok_qty` DOUBLE(17,0) NOT NULL DEFAULT 0,
  `running_time` DECIMAL(32,0) NOT NULL DEFAULT 0,
  `loss_time` DECIMAL(33,0) NOT NULL DEFAULT 0
)""")
	conn.commit()


cur.execute(f"""INSERT INTO {shift_table_name}(department_id,machine_id,mill_date,mill_shift,AR,PR,QR,OEE,pr_by_val,target,actual,sq_mtr,ok_qty,running_time,loss_time)
	SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						ROUND((SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))*100,1) 
					ELSE 0 END AR,
					
					CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) > 100 THEN 100 ELSE ROUND((((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw)))*100,1) END 
					ELSE 0 END AS PR,
					
					CASE WHEN SUM(cp.actual) >0 THEN
						ROUND((SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))*100,1) 
					ELSE 0 END AS QR,
					
					ROUND(((CASE WHEN SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2) >0 THEN 
						(SUM(cp.run_time)/SUM(cp.run_time+cp.loss_time_1+cp.loss_time_2))
					ELSE 0 END)*
					(CASE WHEN (mp.cycle_time_spec*(SUM(cp.run_time)/60)) >0 THEN 
						CASE WHEN (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) > 1 THEN 1 ELSE (((SUM((cp.actual*mp.part_per_stoke)/mp.division_factor))/SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw))) END
					ELSE 0 END)*
					(CASE WHEN SUM(cp.actual) >0 THEN
						(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-cp.rejection)/SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)) 
					ELSE 0 END))*100,1) AS OEE,
					
					SUM(((cp.run_time/60)*((mp.cycle_time_spec)))*mp.draw) AS pr_by_val,
					
					ROUND(SUM(cp.target*mp.draw),0) AS target,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor),0) AS actual,
					ROUND(SUM((cp.actual*mp.part_per_stoke)/mp.division_factor)*mp.width,0) AS sq_mtr,
					
					ROUND(SUM(((cp.actual*mp.part_per_stoke)/mp.division_factor)-(cp.rejection)),0) AS ok_qty,
					SUM(cp.run_time) AS running_time,
					SUM(cp.loss_time_1+cp.loss_time_2) AS loss_time
					
					FROM digital_factory_v1_completed.production_082022 cp,
					digital_factory_v1.master_product mp
			
					WHERE cp.product_id=mp.product_id
					 AND cp.mill_shift={mill_shift} and cp.mill_date="{mill_date}"
					GROUP BY machine_id""")
conn.commit()

cur.execute(f"""DELETE FROM {day_table_name} WHERE mill_date='{mill_date}'""")
conn.commit()
cur.execute(f"""INSERT INTO {day_table_name}(department_id,machine_id,mill_date,mill_shift,AR,PR,QR,OEE,pr_by_val,target,actual,sq_mtr,ok_qty,running_time,loss_time)
			SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					ROUND(AVG(AR),1) AS AR,
					ROUND(AVG(PR),1) AS PR,
					ROUND(AVG(QR),1) AS QR,
					ROUND(AVG(OEE),1) AS OEE,
					SUM(pr_by_val) AS pr_by_val,
					SUM(target) AS target,
					SUM(actual) AS actual,
					SUM(sq_mtr) AS sq_mtr,
					SUM(ok_qty) AS ok_qty,
					SUM(running_time) AS running_time,
					SUM(loss_time) AS loss_time
					FROM {shift_table_name} cp
					WHERE mill_date="{mill_date}"
					GROUP BY mill_date,machine_id""")
conn.commit()

cur.execute(f"""DELETE FROM {month_table_name} WHERE SUBSTRING(mill_date,6,2)={month}""")
conn.commit()
cur.execute(f"""INSERT INTO {month_table_name}(department_id,machine_id,mill_date,mill_shift,AR,PR,QR,OEE,pr_by_val,target,actual,sq_mtr,ok_qty,running_time,loss_time)
    SELECT cp.department_id,cp.machine_id,cp.mill_date,cp.mill_shift,
					ROUND(AVG(AR),1) AS AR,
					
					ROUND(AVG(PR),1) AS PR,
					
					ROUND(AVG(QR),1) AS QR,
					
					ROUND(AVG(OEE),1) AS OEE,
					
					SUM(pr_by_val) AS pr_by_val,
					
					SUM(target) AS target,
					SUM(actual) AS actual,
					SUM(sq_mtr) AS sq_mtr,
					
					SUM(ok_qty) AS ok_qty,
					SUM(running_time) AS running_time,
					SUM(loss_time) AS loss_time
					
					FROM {day_table_name} cp
					WHERE SUBSTRING(mill_date,6,2)={month}""")
conn.commit()
	

	

	
	
